<a href="https://colab.research.google.com/github/XiaohanDeng/MAST30034_Python/blob/main/CapProj-CLUE-Rule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from itertools import combinations

# ===============================
# 加载数据 Read File
# ===============================
gdf = gpd.read_file("vic-lgovt-com-com-clue-floor-use-block-2011-na.json")

print("CRS:", gdf.crs)
print("First 5 rows:")
display(gdf.head())
print("Data types:")
print(gdf.dtypes)


# ===============================
# 1. Geometry Rules
# ===============================

# Rule 1: 检查无效几何
# Invalid Geometry of Polygons (self-intersection,zero area, repeated points)

invalid_geoms = gdf[~gdf.is_valid]
print(f"Invalid geometries: {len(invalid_geoms)}")

# Rule 2: Overlapping Geometry 检查重叠几何
sindex = gdf.sindex
overlaps = []
for idx1, row1 in gdf.iterrows():
    possible_matches = list(sindex.intersection(row1.geometry.bounds))
    for idx2 in possible_matches:
        if idx1 != idx2:
            if row1.geometry.intersects(gdf.iloc[idx2].geometry):
                overlaps.append((idx1, idx2))
print(f"Overlapping feature pairs (first 5): {overlaps[:5]}") #first 5 as examples
print(f"Total overlaps: {len(overlaps)}")


# ===============================
# 2. Attribute Rules
# ===============================

# Rule 3: 检查 NULL 值（confidentiality）
null_counts = gdf.isnull().sum()
print("🔍 Null values per column:\n", null_counts)

# Rule 4: Floor space 必须 ≥ 0
if "floor_space" in gdf.columns:
    negative_values = gdf[gdf["floor_space"] < 0]
    print(f"❌ Negative floor_space count: {len(negative_values)}")

# Rule 5: 检查分项总和与总列一致
if "total" in gdf.columns:
    component_cols = [col for col in gdf.columns if col not in ["geometry", "total"]]
    gdf["component_sum"] = gdf[component_cols].sum(axis=1, skipna=True)
    inconsistent = gdf[gdf["component_sum"] > gdf["total"]]
    print(f"❌ Inconsistent totals (component_sum > total): {len(inconsistent)}")


# ===============================
# 3. Spatial Integrity Rules
# ===============================

# Rule 6: 检查 CRS 是否定义
if gdf.crs is None:
    print("❌ CRS is missing!")
else:
    print(f"✅ CRS defined: {gdf.crs}")


# ===============================
# 4. Business Logic Rules
# ===============================

# Rule 7: 每个 block 的空间用途不应冲突
if "block_id" in gdf.columns and "space_use" in gdf.columns:
    conflict_blocks = gdf.groupby("block_id")["space_use"].nunique()
    conflict_blocks = conflict_blocks[conflict_blocks > 3]  # 自定义阈值，比如超过 3 种用途就算冲突
    print(f"❌ Conflict blocks (>3 different uses): {len(conflict_blocks)}")
